# Sentiment_Analysis_Pierre

In [1]:
import pandas as pd
import numpy as np
import sys
import re
from tqdm import tqdm
import pickle
import copy

## Add cprofile for evaulation of a function's speed

In [2]:
import cProfile,pstats, io
def profile(fct):
    """ a decorator for the function 
        use by writing @profile before any function that needs evaluation"""
    def inner(*args,**kwargs):
        pr = cProfile.Profile()
        pr.enable()
        retval = fct(*args,**kwargs)
        s=i0.StringIO()
        sortBy = 'cumulative'
        ps = pstats.Stats(pr,stream = s).sort_stats(sortBy)
        ps.print_stats()
        print (s.getvalue())
        return retval

## Download Unlabelled articles

In [3]:
import json

raw_json_data = []
with open('./data/20200420_20200714_business_articles.json') as f:
    for line in f:
        raw_json_data.append(json.loads(line))

In [4]:
print ("data type",type (raw_json_data))
print ("json",type (raw_json_data[0]))
print ("keys",raw_json_data[0].keys())
print ("length", len(raw_json_data))
#print (raw_json_data[0])

data type <class 'list'>
json <class 'dict'>
keys dict_keys(['published', 'link', 'message', 'Feed', 'title', '@version', 'author', '@timestamp', 'full-text', 'type'])
length 416307


## Extracting url, title & full_text of each article:

In [5]:
urls = list()
plain_texts = list()
titles = list()
labels = list()

min_article_size = 2000
for article in raw_json_data:
    plain_text = article.get('full-text')
    title = article.get('title')
    url = article.get('link')
    if (plain_text and "Article `download()` failed" != plain_text[:27] and "Please enable cookies" != plain_text[:21] and len(plain_text)>min_article_size):
        plain_texts.append(plain_text)
        urls.append(url)
        titles.append(title)
        labels.append(list())
       

## Build DataFrame with extacted data

In [6]:
#Statistics
# 358192 removing "Article `download()` failed" 
# 340987 removing "Article `download()` failed" and "Please enable cookies"
# 215039 removing "Article `download()` failed" and "Please enable cookies" and size<min_article_size = 2000
data = np.array([urls,titles, plain_texts, labels]).T
columns=["url", "title", "plain_text", "label"]
df_articles = pd.DataFrame(data=data, columns=columns)

/opt/anaconda3/envs/P-SAT/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [7]:
df_articles.head()

,url,title,plain_text,label
0,https://www.dailymail.co.uk/tvshowbiz/article-...,MasterChef's Harry Foster hits back at claims ...,Eliminated MasterChef contestant Harry Foster ...,[]
1,https://www.washingtontimes.com/news/2020/jun/...,"Protest arrests logjam tests NYC legal system,...",NEW YORK (AP) - A wave of arrests in the New Y...,[]
2,https://www.dailymail.co.uk/news/article-83114...,Labour's Anneliese Dodds says she will REFUSE ...,A top shadow minister today said there was not...,[]
3,http://feeds.reuters.com/~r/Reuters/worldNews/...,Civil unrest rages in Minneapolis over raciall...,MINNEAPOLIS (Reuters) - Peaceful rallies gave ...,[]
4,https://www.dailymail.co.uk/news/article-82734...,Australia 'beats the cr*p' out of coronavirus ...,Australia is 'beating the c**p' out of coronav...,[]


# POS TAGGING

In [8]:
# Imports
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pierre/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
paragraph = df_articles["plain_text"][0]
sent_text = nltk.sent_tokenize(paragraph) # this gives us a list of sentences
# Now Loop over each sentence and tokenize it separately
tokenized_paragraph = list()
for sentence in sent_text:
    tokenized_text = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokenized_text)
    tokenized_paragraph.append(tagged)
    #print(tagged)
tokenized_paragraph[0]

[('Eliminated', 'VBN'),
 ('MasterChef', 'NNP'),
 ('contestant', 'JJ'),
 ('Harry', 'NNP'),
 ('Foster', 'NNP'),
 ('has', 'VBZ'),
 ('hit', 'VBN'),
 ('back', 'RB'),
 ('at', 'IN'),
 ('unfair', 'JJ'),
 ('criticism', 'NN'),
 ('against', 'IN'),
 ('judge', 'NN'),
 ('Melissa', 'NNP'),
 ('Leong', 'NNP'),
 ('.', '.')]

In [ ]:
# wordNet
# SynNet
# imageNet
